In [ ]:
from langchain_aws import ChatBedrock
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import json
import random
from multiprocessing import Pool
import boto3

In [ ]:
# reference to the foundation model via amazon bedrock.
model = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # claude is awesome!    
    model_kwargs={"temperature": 1.0,
                 'max_tokens': 20000}                  # parameters vary by model
)

In [ ]:
# returns template text for passing into LLM
# a palindrome is a word or phrase that is identical when spelled backward, excluding punctuation.
prompt = PromptTemplate(
    template="""
context: ticketmom provides online ticketing for large scale events such as concerts,
broadway shows, ballet, opera, car races, monster truck rallies, kid's shows, and festivals.
ticketmom customers are generally polite, but sometimes are impatient or frustrated in a way
that makes it difficult to work with them.

role: you are a ticketmom customer service agent who works with customers 
via email and telephone to resolve issues with their orders and accounts.
you are polite, but sometimes get frustrated with difficult situations.

ticketing event: {event_name} in {city} at the venue: {venue} on {event_datetime}

generate notes describing a complicated interaction with a customer who bought tickets for this event.

format: free form text. the length of the notes should reflect the complexity of the issue.
"""
)

In [ ]:
events = PromptTemplate(
    template="""
context: ticketmom provides online ticketing for large scale events such as concerts,
broadway shows, ballet, opera, car races, monster truck rallies, kid's shows, and festivals.

create a list of 50 fictitious events which would be sold via ticketmom.

return the event data only as a list of JSON objects with the following structure:

"event_name": EVENT_NAME, "city", EVENT_CITY, "event_datetime": EVENT_DATETIME, "venue": EVENT_VENUE
"""
)

In [ ]:
events_chain = events | model | StrOutputParser()

In [ ]:
# prompt is passed into the model. response is parsed for easy reading.
chain = prompt | model | StrOutputParser()

In [ ]:
events_response = events_chain.invoke({})
event_list = json.loads(events_response)

In [ ]:
def event_document(event, response):
    e = event
    e['incident_notes'] = response
    return e

def doc_to_text(event_doc):
    return f"""
------------------------
event: {event_doc['event_name']}

city: {event_doc['city']}

venue: {event_doc['venue']}

event_date: {event_doc['event_datetime']}

customer service notes:

{event_doc['incident_notes']}

"""

In [ ]:
for event in event_list:
    event_docs = ""
    for i in range(0, random.randint(2, 4)):
        cs_notes = chain.invoke(event)
        this_event_doc = event_document(event, cs_notes)
        event_docs += doc_to_text(this_event_doc)
    print(event_docs)


In [ ]:
def generate_interaction(event):
    try:
        cs_notes = chain.invoke(event)
        this_event_doc = doc_to_markdown(event_document(event, cs_notes))
        return this_event_doc
    except e as Exception:
        return 'ERROR'

s3_client = boto3.client('s3', region_name='us-west-2')

for event in event_list:
    this_event_interactions = []
    with Pool(8) as p:
        this_event_interactions = p.map(generate_interaction, [event_list[0] for x in range(8, random.randint(9, 150))])
        joined = ''.join(this_event_interactions)
        s3_client.put_object(
            Bucket='ticketmom-cs-interactions',
            Key=event['event_name'] + '.txt',
            Body=bytes(joined,'utf-8')
        )
        print(event)
